In [ ]:
import numpy as np
import pandas as pd

### Reading the file
Five attributes are extracted for each domain: protein name, family name, e-value, initial and final position in the alignment.

In [ ]:
target_names=[]
query_names=[]
e_values=[]
start=[]
end=[]
filename="hits_same_clan.scanned"
with open(filename) as file:
    for i, line in enumerate(file):
        if i<3:
            continue
        line=line.split()
        target_names.append(line[0])
        query_names.append(line[3])
        e_values.append(line[6])
        start.append(line[17])
        end.append(line[18])

In [ ]:
target_names=np.array(target_names)
query_names=np.array(query_names)
e_values=np.array(e_values)
start=np.array(start)
end=np.array(end)

### DataFrame creation and match definition
Two domains match if the size of their intersection is greater than $0.8$ times the size of their union (matching in terms of proteins and families is checked elsewhere, so it is not taken into account here).

In [ ]:
families=np.unique(query_names)
family_index={family: np.where(families==family)[0][0] for family in families}
family_sizes=np.zeros_like(families, dtype=np.int32)
data={
    "protein": target_names,
    "family": query_names,
    "evalue": e_values,
    "start": start,
    "end":end
}
df=pd.DataFrame(data)
def intersection_size(start1, start2, end1, end2):
    return len(range(max(start1, start2), min(end1, end2)))
def union_size(start1, start2, end1, end2):
    return len(range(start1, end1))+len(range(start2, end2))-intersection_size(start1, start2, end1, end2)
def match(row1, row2):
    start1=int(row1[3])
    start2=int(row2[3])
    end1=int(row1[4])
    end2=int(row2[4])
    if intersection_size(start1, start2, end1, end2)/union_size(start1, start2, end1, end2)>0.8:
        return True
    return False

A matrix initially filled with zeros will keep track of matches in the dataset. For each couple of matching families, the tensor `ev_tensor` keeps track of the minimum e-values in matching proteins.

In [ ]:
matches=np.zeros((len(families), len(families)))
ev_tensor=np.ones((len(families), len(families), 2))

### Match checking algorithm
In order to make match checking efficient, data are sorted by protein name. In fact, only domains found in the same protein may be matching and it is costly to scan the whole dataset to look for other instances of the same protein.  
For each entry $i$, $j$ is initialized to be equal to $i+1$ and then increased by $1$ each time the same protein of position $i$ is found. If there is a match (same protein, different family and overlap>$0.8$) between lines $i$ and $j$, the entry of the matrix corresponding to their families is increased by $1$ and the tensor that stores minimum e-values of this couple of families is updated (if needed).
When the $j$-th protein becomes different from the $i$-th, $i$ is increased by $1$.  
Moreover, while scanning the dataset, sizes (numbers of representatives) of families are computed.


In [ ]:
df=df.sort_values("protein", ignore_index=True)

In [ ]:
i=0
n=len(df)
data=df.to_numpy()
for i in range(n-1):
    print(str(i)+'/'+str(n), end='\r')
    ith_row=data[i]
    protein=ith_row[0]
    family1=ith_row[1]
    evalue1=float(ith_row[2])
    family_sizes[family_index[family1]]+=1
    j=i+1
    jth_row=data[j]
    while jth_row[0]==protein:
        family2=jth_row[1]
        evalue2=float(jth_row[2])
        if family2!=family1 and match(ith_row, jth_row):
            fam1index=family_index[family1]
            fam2index=family_index[family2]
            matches[fam1index, fam2index]+=1
            if min(fam1index, fam2index)==fam1index:
                ev_tensor[fam1index, fam2index, 0]=min(evalue1, ev_tensor[fam1index, fam2index, 0])
                ev_tensor[fam1index, fam2index, 1]=min(evalue2, ev_tensor[fam1index, fam2index, 1])
            else:
                ev_tensor[fam2index, fam1index, 0]=min(evalue2, ev_tensor[fam2index, fam1index, 0])
                ev_tensor[fam2index, fam1index, 1]=min(evalue1, ev_tensor[fam2index, fam1index, 1])
        j+=1
        if j==n:
            break
        jth_row=data[j]

In [ ]:
print(family_sizes)

[ 12508    614    366  22306  34873   1204    392  67600    646    165
    198    545   1649  27480   4013 125899    394  11770    796   2684
  45075   1305   1116   9505  63477   3784    148   7550    695 265255
  12822   3105   2661   6997   1330    299   1929  17132   6211    932
  24684   3358  23124    382   2586   2095    144   9741   1116  27028
   1337  67564  15440    388   1223  23381   9242   7874   2135    360
   3043   1186  87846   1842    390    959    944  76214    363  11568
  17812  35921 202030   1101    859   1185  74319    345    209]


The final matrix has to be symmetrical, but the insertion of $i,j$-th element has been made partly on $i,j$ itself and partly on $j,i$, so the two values have to be summed.  
In addition, values should be rescaled by a factor that keeps track of the number of possible matches, at the moment set to the minimum size of families $i$ and $j$.

In [ ]:
normalized_matrix=np.zeros_like(matches)
for i in range(len(families)):
    for j in range(i,len(families)):
        matches[i, j]+=matches[j, i]
        normalized_matrix[i, j]/=min(family_sizes[i], family_sizes[j])
        matches[j, i]=matches[i, j]
        normalized_matrix[j,i]=normalized_matrix[i,j]

### Match elimination

Families $0$ and $1$ have $2$ matches, with minimal evalues set respectively to $0.01$ and $0.0016$.

In [ ]:
print(matches[0,1])
print(families[0], families[1])
print(ev_tensor[0,1])

2.0
MC10877_cdhit.fasta MC118196_cdhit.fasta
[0.01   0.0016]


`ev_tensor` stores the minima for matches between two families: then the maximum of these two values is taken for the corresponding family as a cutoff level. However, since matches may happen between multiple families, pairwise comparison is not sufficient. The threshold for a certain family is set to the minimum threshold required by pairwise comparison between families.

In [ ]:
cutoffs=np.ones_like(families, dtype=np.float64)
for i in range(len(families)):
    for j in range(i,len(families)):
        if np.argmax(ev_tensor[i,j])==0:
            cutoffs[i]=min(cutoffs[i], ev_tensor[i,j,0])
        else:
            cutoffs[j]=min(cutoffs[j], ev_tensor[i,j,1])
print(cutoffs)

[2.2e-022 8.7e-012 1.5e-050 1.1e-097 3.5e-135 2.0e-009 7.6e-009 2.2e-107
 1.8e-006 1.0e+000 1.0e+000 3.0e-081 1.0e+000 9.9e-071 5.2e-023 7.6e-287
 2.1e-014 2.3e-040 3.2e-006 7.1e-023 1.8e-066 8.6e-010 1.0e-004 2.3e-041
 0.0e+000 1.3e-039 1.0e+000 1.7e-069 5.0e-008 0.0e+000 1.1e-030 1.0e-014
 3.5e-021 1.5e-080 8.2e-018 1.0e+000 1.9e-016 9.1e-055 8.2e-007 1.0e+000
 6.8e-073 1.5e-051 8.4e-064 1.0e+000 1.6e-039 5.1e-029 1.0e+000 1.1e-046
 6.9e-033 5.6e-119 2.5e-012 9.4e-222 1.0e-041 1.8e-002 1.5e-019 8.6e-058
 2.5e-030 5.0e-063 7.9e-040 1.8e-022 2.8e-021 7.5e-042 1.0e-197 8.1e-036
 1.2e-026 1.3e-016 4.4e-061 2.4e-101 1.0e+000 1.3e-055 1.0e-074 9.8e-084
 8.4e-011 1.0e+000 1.0e+000 3.3e-015 3.0e-167 1.0e+000 1.0e+000]


In [ ]:
df2=df

Here we are deleting the datapoints belonging to a certain family with e-value lower than the threshold value.

In [ ]:
for family in families:
    df2=df2.loc[(df2['family'] != family) | (df2['evalue'].apply(lambda x: float(x)) < cutoffs[family_index[family]])]

In [ ]:
print("Size of dataset without matches: ", len(df2))
print("Size of the original dataset: ", len(df))

Size of dataset without matches:  175151
Size of the original dataset:  1508769


In [ ]:
new_matches=np.zeros((len(families), len(families)))

In [ ]:
df2=df2.sort_values("protein", ignore_index=True)

Match checking on the new dataset to make sure there are no matches.

In [ ]:
i=0
n=len(df2)
data=df2.to_numpy()
for i in range(n-1):
    print(str(i)+'/'+str(n), end='\r')
    ith_row=data[i]
    protein=ith_row[0]
    family1=ith_row[1]
    evalue1=float(ith_row[2])
    family_sizes[family_index[family1]]+=1
    j=i+1
    jth_row=data[j]
    while jth_row[0]==protein:
        family2=jth_row[1]
        evalue2=float(jth_row[2])
        if family2!=family1 and match(ith_row, jth_row):
            fam1index=family_index[family1]
            fam2index=family_index[family2]
            print(ith_row,fam1index, jth_row, fam2index)
            new_matches[fam1index, fam2index]+=1
        j+=1
        if j==n:
            break
        jth_row=data[j]

In [ ]:
print("Matches in new dataset: ", np.sum(new_matches))
print("Matches in original dataset: ", np.sum(np.triu(matches)))

Matches in new dataset:  0.0
Matches in original dataset:  830542.0
